https://dev.evernote.com/doc/articles/authentication.php

Let's draw from https://requests-oauthlib.readthedocs.org/en/latest/examples/bitbucket.html (BitBucket also uses OAuth 1.0a, right?)

In [ ]:
from evernote.edam.notestore.ttypes import (NoteFilter,
                                            NotesMetadataResultSpec
                                           )
from settings import (EVERNOTE_CLIENT_ID, EVERNOTE_CLIENT_SECRET, EVERNOTE_SANDBOX)
import requests_oauthlib

In [ ]:
# Credentials you get from adding a new consumer in bitbucket -> manage account
# -> integrated applications.

key = EVERNOTE_CLIENT_ID
secret = EVERNOTE_CLIENT_SECRET
sandbox = EVERNOTE_SANDBOX

In [ ]:
# OAuth endpoints given in the Evernote API documentation
# https://dev.evernote.com/doc/articles/authentication.php

# https://sandbox.evernote.com/oauth
# https://sandbox.evernote.com/oauth?oauth_callback=http://www.foo.com&oauth_consumer_key=sample-api-key-4121&oauth_nonce=3166905818410889691&oauth_signature=T0+xCYjTiyz7GZiElg1uQaHGQ6I=&oauth_signature_method=HMAC-SHA1&oauth_timestamp=1429565574&oauth_version=1.0

#request_token_url = 'https://bitbucket.org/!api/1.0/oauth/request_token'


BASE_URL = "https://www.evernote.com" if not sandbox  \
           else "https://sandbox.evernote.com"
    

request_token_url = '{}/oauth'.format(BASE_URL)

# authorization_base_url = 'https://bitbucket.org/!api/1.0/oauth/authenticate'
authorization_base_url = '{}/OAuth.action'.format(BASE_URL)

access_token_url = '{}/oauth'.format(BASE_URL)

In [ ]:
# 2. Fetch a request token
from requests_oauthlib import OAuth1Session
evernote = OAuth1Session(key, client_secret=secret,
         callback_uri='http://127.0.0.1:5000/callback')
fetch_response = evernote.fetch_request_token(request_token_url)

resource_owner_key = fetch_response.get('oauth_token')
resource_owner_secret = fetch_response.get('oauth_token_secret')


In [ ]:
# 3. Redirect user to Evernote for authorization
authorization_url = evernote.authorization_url(authorization_base_url)
print ('Please go here and authorize,', authorization_url)

In [ ]:
# 4. Get the authorization verifier code from the callback url
redirect_response = raw_input('Paste the full redirect URL here:')
oauth_response = evernote.parse_authorization_response(redirect_response)
verifier = oauth_response.get('oauth_verifier')

In [ ]:
# instantiate a new OAuth1Session (to model how we'd do this in a web app)

evernote2 = OAuth1Session(key,
                          client_secret=secret,
                          resource_owner_key=resource_owner_key,
                          resource_owner_secret=resource_owner_secret)

token_dict = evernote.fetch_access_token(access_token_url)

In [ ]:
# 5. Fetch the access token
#token_dict = evernote.fetch_access_token(access_token_url)
#token_dict

In [ ]:
# 6. Fetch a protected resource, i.e. user profile
# https://github.com/evernote/evernote-sdk-python/blob/master/sample/client/EDAMTest.py

import hashlib
import binascii
import evernote.edam.userstore.constants as UserStoreConstants
import evernote.edam.type.ttypes as Types

from evernote.api.client import EvernoteClient

auth_token = token_dict[u'oauth_token']
client = EvernoteClient(token=auth_token, sandbox=sandbox)

user_store = client.get_user_store()

version_ok = user_store.checkVersion(
    "Evernote EDAMTest (Python)",
    UserStoreConstants.EDAM_VERSION_MAJOR,
    UserStoreConstants.EDAM_VERSION_MINOR
)
print "Is my Evernote API version up to date? ", str(version_ok)
print ""
if not version_ok:
    exit(1)

note_store = client.get_note_store()

# List all of the notebooks in the user's account
notebooks = note_store.listNotebooks()

In [ ]:
[(notebook.name, notebook.guid) for notebook in notebooks]

In [ ]:
# now get number of notes in a notebook

OSF_NOTEBOOK_GUID = '014e8f86-110e-4745-bfee-4b21d8c6b51d'
counts = note_store.findNoteCounts(NoteFilter(), False)
counts.notebookCounts.get(OSF_NOTEBOOK_GUID)

In [ ]:
# get notes metadata from notebook

def notes_metadata(client, **input_kw):
    """ """
    # http://dev.evernote.com/documentation/reference/NoteStore.html#Fn_NoteStore_findNotesMetadata

    noteStore = client.get_note_store()
    
    # pull out offset and page_size value if supplied
    offset = input_kw.pop("offset", 0)
    page_size = input_kw.pop("page_size", 100)

    # let's update any keywords that are updated
    # http://dev.evernote.com/documentation/reference/NoteStore.html#Struct_NotesMetadataResultSpec

    include_kw = {
        'includeTitle':False,
        'includeContentLength':False,
        'includeCreated':False,
        'includeUpdated':False,
        'includeDeleted':False,
        'includeUpdateSequenceNum':False,
        'includeNotebookGuid':False,
        'includeTagGuids':False,
        'includeAttributes':False,
        'includeLargestResourceMime':False,
        'includeLargestResourceSize':False
    }

    include_kw.update([(k, input_kw[k]) for k in set(input_kw.keys()) & set(include_kw.keys())])

    # keywords aimed at NoteFilter
    # http://dev.evernote.com/documentation/reference/NoteStore.html#Struct_NoteFilter
    filter_kw_list = ('order', 'ascending', 'words', 'notebookGuid', 'tagGuids', 'timeZone', 'inactive', 'emphasized')
    filter_kw = dict([(k, input_kw[k]) for k in set(filter_kw_list) & set(input_kw.keys())])

    # what possible parameters are aimed at NoteFilter
    #order	i32		optional
    #ascending	bool		optional
    #words	string		optional
    #notebookGuid	Types.Guid		optional
    #tagGuids	list<Types.Guid>		optional
    #timeZone	string		optional
    #inactive   bool
    #emphasized string

    more_nm = True

    while more_nm:

        # grab a page of data
        note_meta = noteStore.findNotesMetadata(NoteFilter(**filter_kw), offset, page_size,
                                    NotesMetadataResultSpec(**include_kw))

        # yield each individually
        for nm in note_meta.notes:
            yield nm

        # grab next page if there is more to grab
        if len(note_meta.notes):
            offset += len(note_meta.notes)
        else:
            more_nm = False

In [ ]:
notes = list(notes_metadata(client, notebookGuid=OSF_NOTEBOOK_GUID, 
                    includeTitle=True, 
                    includeUpdated=True,
                    includeCreated=True))

In [ ]:
notes[0].title, notes[0].guid